In [35]:
import os
import numpy as np
import csv
import torch
import sys
from datetime import datetime
from tqdm import tqdm
import pickle

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def readCSV(fname):
    file = open(f"{os.getcwd()}/{fname}.csv")
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    
    rows = []
    for row in csvreader:
        rows.append(row)
    file.close()
        
    return header, rows

In [4]:
coll_head, coll_rows = readCSV("Cleaning_collections")
api_head, api_rows = readCSV("Data_API")

In [30]:
len(api_rows)

6071027

In [5]:
api_rows[0]

['',
 '87094722.0',
 '0xed9ff1c311a611e5c457ce0a49bbbb7ed55efca988421aee27a01a8f8f6810f2',
 '0x76481caa104b5f6bccb540dae4cefaf1c398ebea',
 '',
 '0xe0fb7622091e3d9ef9b438471b10b9ea88c7cf6b',
 '',
 'https://card.godsunchained.com/?id=33&q=4',
 '',
 '',
 '',
 '0.0002',
 'ETH',
 '0.030318',
 '',
 '',
 'Godsunchained',
 'Godsunchained',
 '2019-11-30 00:00:00',
 '2019-11-30 23:59:33',
 '',
 "('Godsunchained', '87094722.0')",
 'Godsunchained',
 'Games']

In [6]:
api_head

['Smart_contract',
 'ID_token',
 'Transaction_hash',
 'Seller_address',
 'Seller_username',
 'Buyer_address',
 'Buyer_username',
 'Image_url_1',
 'Image_url_2',
 'Image_url_3',
 'Image_url_4',
 'Price_Crypto',
 'Crypto',
 'Price_USD',
 'Name',
 'Description',
 'Collection',
 'Market',
 'Datetime_updated',
 'Datetime_updated_seconds',
 'Permanent_link',
 'Unique_id_collection',
 'Collection_cleaned',
 'Category']

In [22]:
el = api_rows[53]
d1 = datetime.fromisoformat(el[18]).timestamp()
d2 = datetime.fromisoformat(el[19])
coll = el[16]

In [24]:
coll

'Godsunchained'

In [31]:
data = {}

for i, rowi in tqdm(enumerate(api_rows)):
    coll = rowi[22]
    if not coll in data.keys():
        data[coll] = [i]
    else:
        data[coll] = data[coll] + [i]

2657173it [44:02, 1005.66it/s] 


KeyboardInterrupt: 

In [33]:
collections = set([row[22] for row in api_rows])

In [34]:
len(collections)

4624

In [37]:
for c in tqdm(collections):
    nfts = [row for row in api_rows if row[22] == c]
    with open(f"collections/{c}", "wb") as f:
        pickle.dump(nfts, f)

100%|███████████████████████████████████████| 4624/4624 [41:18<00:00,  1.87it/s]


# Get All Dates

In [69]:
c = list(collections)[1000]

In [70]:
c_nfts = pickle.load(open(f"collections/{c}", "rb"))

In [ ]:
# USD_mean, USD_median, USD_var, USD_sum, USD_prod
# CRYTO_mean, CRYPTO_var, CRYPTO_median, CRYPTO_sum, CRYPTO_prod, 
# CRYPTO (1=WAX), COUNT, Buyer Address, Smart Contract, Category

coll_stats = np.zeros((11))

In [97]:
def collStats(c, start, end):
    doc_vars = [[]]*6
    
    c_nfts = pickle.load(open(f"collections/{c}","rb"))
    print(c_nfts)
    
    for nft in c_nfts:
        doc_vars[0] = doc_vars[0] + [nft[13]] # USD
        doc_vars[1] = doc_vars[1] + [nft[11]] # CRYPTO
        doc_vars[2] = doc_vars[2] + [nft[12]] # COIN
        doc_vars[3] = doc_vars[3] + [nft[5]]  # Buyer address
        doc_vars[4] = doc_vars[4] + [nft[17]] # Market
        doc_vars[5] = doc_vars[5] + [nft[23]] # Category
    
    output = []
    for el in doc_vars[0:2]:
        el = np.array(el).astype(float)
        output.append([np.mean(el), np.median(el), np.var(el), np.sum(el), np.prod(el)])
    output.append(doc_vars[2])
    output.append(doc_vars[3])
    output.append(doc_vars[4])
    output.append(doc_vars[5])
    return output
              

In [96]:
stats = collStats(c, 0, 0)
stats

[[0.5566265714285714,
  0.26135,
  0.5231295218076734,
  3.8963859999999997,
  0.000741944386524028],
 [1.9985714285714287, 1.0, 5.982869387755102, 13.99, 7.99],
 ['WAX', 'WAX', 'WAX', 'WAX', 'WAX', 'WAX', 'WAX'],
 ['boab.wam',
  '1mjqw.wam',
  'boab.wam',
  '1mjqw.wam',
  'mvsb.wam',
  '1mjqw.wam',
  'evsau.wam'],
 ['Atomic', 'Atomic', 'Atomic', 'Atomic', 'Atomic', 'Atomic', 'Atomic'],
 ['Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other']]

In [72]:
dates = [nft[19] for nft in c_nfts]
print(dates)

['2021-04-05 01:29:33', '2021-04-05 01:29:21', '2021-04-05 01:28:24', '2021-04-05 01:28:16', '2021-04-05 01:26:36', '2021-04-05 01:26:28', '2021-04-03 13:24:13']


In [46]:
nft_ids = set([nft[1] for nft in nfts])
print(len(nft_ids))
print(len(nfts))
#print(nft_ids)
prices = {}

for id_str in nft_ids:
    prices[id_str] = 0

1482
1535


# Get All Collection Stats For Each Date

# Create function to get stats of an nft's given collection

# Iterate Over Collections - Parse Sliding Windows

### Data to be included (for each date) for the NFT:
- All collection stats (number of items, volume traded)
- NFT collection stats (number of items, volume traded)
- NFT stats (crypto, price USD, price cypto, category)
- Label - percentage return